In [32]:
import pandas as pd
import xlrd
import numpy as np

In [8]:
atencion = pd.read_excel('ATENCIONES_D.xlsx',sheet_name='Hoja1')
inscripcion = pd.read_excel('INSCRIPCIONES_D.xlsx',sheet_name='INSCRITOS')

In [9]:
inscripcion.head()

,COMUNA,Ficho_Inscripcion,Fecha_inscrip,Dire_com,Tip_Doc,ID,Fechanac,Edad,Genero,Barrio,...,Tiem_gafas,Glaucoma,Enfermedad,Descripcion,SISBEN,Grupo Edad,COD_BARRIO,Unnamed: 23,Categ,Ctd
0,70,1,43577,CASA DE GOBIERNO ALTAVISTA,CC,22023886,1959-04-17,60.0,M,ALTAVISTA,...,Mas de 2 años,Si,G,TENGO UNA BOLA EN EL PARPADO SUPERIOR IZQUIERD...,Si,MAYOR DE 60,7084,0,NO ATENDER,1
1,70,2,43577,CASA DE GOBIERNO ALTAVISTA,CC,3566898,1953-03-21,66.0,H,LA ESPERANZA,...,NaN,NaN,NaN,NaN,NaN,MAYOR DE 60,7083,3566898,ATENDER,1
2,70,3,43577,CASA DE GOBIERNO ALTAVISTA,CC,3651220,1954-08-24,64.0,H,EL CONSEJO,...,Mas de 2 años,NaN,D,ME ARDEN Y NO PUEDO VER,Si,MAYOR DE 60,7005,3651220,ATENDER,1
3,70,4,43577,CASA DE GOBIERNO ALTAVISTA,CC,70102546,1957-09-03,61.0,H,NaN,...,Mas de 2 años,No,NaN,"NO VEO, SE ME NUBLAN LOS OJOS Y ME ARDEN",No,MAYOR DE 60,7087,70102546,ATENDER,1
4,70,5,43577,CASA DE GOBIERNO ALTAVISTA,CC,32400950,1942-10-06,76.0,M,NaN,...,Mas de 2 años,No,H,NaN,Si,MAYOR DE 60,7087,0,NO ATENDER,1


In [10]:
atencion.head()

,FICHO,CONSECUTIVO,ID,EDAD,GRUPO EDAD,OPTOMETRA,FECHA ATENCION\nCONSULTA,CONSULTA,ESTUCHE Y PAÑO,MONTURA,...,7\nLentes bifocales flat top policarbonato,8\nLentes bifocales invisible CR-39 terminados,9\nLentes bifocales invisible CR-39 tallados,10\nLentes bifocales invisible policarbonato,11\nLentes progresivos en CR-39,12\nLentes progresivos en policarbonato,TOTAL LENTES UNITARIOS,REFERENCIA MONTURA,VALOR USUARIO,Unnamed: 38
0,1,44402.0,4.277358e+07,51.0,ENTRE 18 Y 59,2,2019-05-13,1.0,1.0,1.0,...,0,0,0,0,0,0,2.0,DG5676,99020,70
1,2,44351.0,1.152223e+09,20.0,ENTRE 18 Y 59,1,2019-05-13,1.0,1.0,1.0,...,0,0,0,0,0,0,2.0,ERA2183,110254,70
2,3,44451.0,4.320362e+07,39.0,ENTRE 18 Y 59,3,2019-05-13,1.0,1.0,1.0,...,0,0,0,0,0,0,2.0,FLA1530,105112,70
3,4,44403.0,4.355560e+07,59.0,ENTRE 18 Y 59,2,2019-05-13,1.0,1.0,1.0,...,0,0,0,0,0,2,2.0,WF135,235478,70
4,5,44352.0,1.037618e+09,27.0,ENTRE 18 Y 59,1,2019-05-13,1.0,1.0,1.0,...,0,0,0,0,0,0,2.0,DA668,99020,70


In [30]:
optometria = atencion.merge(inscripcion, left_on = 'ID', right_on = 'ID', how = 'left')

In [31]:
optometria.describe()

,CONSECUTIVO,ID,EDAD,CONSULTA,ESTUCHE Y PAÑO,MONTURA,EJE-OD,2\nLentes monofocales en CR-39 tallados,3\nLentes monofocales en policarbonato terminados\n,5\nLentes bifocales flat top CR-39 terminados,...,10\nLentes bifocales invisible policarbonato,11\nLentes progresivos en CR-39,12\nLentes progresivos en policarbonato,TOTAL LENTES UNITARIOS,VALOR USUARIO,Unnamed: 38,COMUNA,Edad,Unnamed: 23,Ctd
count,8149.000000,8.149000e+03,8149.000000,8149.0,7792.0,7792.0,4622.000000,8144.000000,8144.000000,8151.000000,...,8151.000000,8151.000000,8151.000000,8147.000000,8151.000000,8151.000000,7835.000000,7832.000000,7.835000e+03,7835.0
mean,46945.464106,5.811139e+12,47.994723,1.0,1.0,1.0,68.307984,0.035363,0.108546,0.022574,...,0.119740,0.486566,0.402282,1.912115,150643.336891,23.723838,24.148692,48.241701,6.023104e+12,1.0
std,2799.534425,7.251446e+13,14.589190,0.0,0.0,0.0,51.530083,0.235553,0.443276,0.195610,...,0.474521,0.857896,0.801679,0.409960,60105.064583,21.322529,21.631484,14.546591,7.393404e+13,0.0
min,4545.000000,1.919140e+05,0.000000,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,11.000000,18.000000,1.919140e+05,1.0
25%,44576.000000,4.268571e+07,39.000000,1.0,1.0,1.0,15.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.000000,99020.000000,13.000000,13.000000,39.000000,4.202333e+07,1.0
50%,47148.000000,4.353933e+07,51.000000,1.0,1.0,1.0,80.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.000000,135342.000000,13.000000,13.000000,51.000000,4.353290e+07,1.0
75%,49212.000000,7.173814e+07,57.000000,1.0,1.0,1.0,95.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.000000,190226.000000,15.000000,15.000000,57.000000,7.172321e+07,1.0
max,61977.000000,9.636632e+14,99.000000,1.0,1.0,1.0,180.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,235478.000000,70.000000,70.000000,99.000000,9.636632e+14,1.0


In [42]:
optometria['Astigmatismo'] = np.where(optometria['DX PPAL CIE 10'] == 'H522',1,0)

Eliminacion de acuerdo al juicio del experto

In [52]:
columnas = ['EDAD','CONSULTA','ESTUCHE Y PAÑO','MONTURA', 'DX\nPPAL\nCIE 10','ESFERA-OD','CILINDRO-OD','EJE-OD','ADD-OD','TIPO LENTE-OD','ESFERA-OI','CILINDRO-OI','EJE-OI','ADD-OI','TIPO LENTE-OI','Genero','Usa_Gafas','Glaucoma','Enfermedad', 'ocupacion','Dir_Res','poblacion','etnia','Exam_ojos','Usa_Gafas','Tiem_gafas','Descripcion','SISBEN','COD_BARRIO','Astigmatismo']

In [53]:
optometria_dep = optometria[columnas]

In [54]:
optometria_dep.head()

,EDAD,CONSULTA,ESTUCHE Y PAÑO,MONTURA,DX\nPPAL\nCIE 10,ESFERA-OD,CILINDRO-OD,EJE-OD,ADD-OD,TIPO LENTE-OD,...,Dir_Res,poblacion,etnia,Exam_ojos,Usa_Gafas,Tiem_gafas,Descripcion,SISBEN,COD_BARRIO,Astigmatismo
0,51.0,1.0,1.0,1.0,H524,2,-0.5,100.0,NaN,1,...,CL 34B 125 42,NaN,Ninguna de las anteriores,De 1 a 2 años,Si,Mas de 2 años,"DIFICULTAD PARA VER DE CERCA, MOLESTIA A LA LUZ",Si,7088,0
1,20.0,1.0,1.0,1.0,H522,N,-0.75,70.0,NaN,3,...,CL 34B 125 42,Reinsertado,Ninguna de las anteriores,De 1 a 2 años,Si,Mas de 2 años,"BORROSO PARA VER DE LEJOS, MOLESTIA CON MUCHA ...",Si,7088,1
2,39.0,1.0,1.0,1.0,H522,-1,-2.5,5.0,NaN,2,...,CR 130 34B 98(INTERIOR 111),NaN,Ninguna de las anteriores,De 1 a 2 años,Si,Mas de 2 años,NO VEO DE CERCA NI DE LEJOS,Si,7088,1
3,59.0,1.0,1.0,1.0,H524,1.5,NaN,NaN,2.75,12,...,CR 125 34AA 56,Madre cabeza de familia,Ninguna de las anteriores,Mas de 2 años,Si,Mas de 2 años,VEO OPACO,No,7088,0
4,27.0,1.0,1.0,1.0,H522,-0.25,NaN,NaN,NaN,1,...,CR 116 34 126(INTERIOR 191),Ninguna de las anteriores,Ninguna de las anteriores,Mas de 2 años,Si,Mas de 2 años,"ME ARDEN LOS OJOS, VEO BORROSO DE CERCA, SE ME...",Si,7088,1


In [68]:
optometria_dep['Requiere_Gafas'] = np.where((optometria_dep['ESTUCHE Y PAÑO'] == 1) & (optometria_dep['MONTURA'] == 1) ,1,0) #1 Tiene gafas, 0 no tiene

/Users/yosel/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
optometria_dep['Hipertension'] = optometria_dep['Enfermedad'].map(lambda x: str(x).upper().find('H')>0)
optometria_dep['Diabetes'] = optometria_dep['Enfermedad'].map(lambda x: str(x).upper().find('D')>0)
optometria_dep['Glaucoma'] = optometria_dep['Enfermedad'].map(lambda x: str(x).upper().find('G')>0)

/Users/yosel/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/yosel/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yosel/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde